## Loading Data

In [109]:
import os
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import date

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
i2m = list(zip(range(1,13), ['Gener', 'Febrer', 'Marc', 'Abril', 'Maig', 'Juny', 'Juliol', 'Agost', 'Setembre', 'Octubre', 'Novembre', 'Desembre']))
for year in [2023, 2022, 2021, 2020, 2019]:
    for month, month_name in i2m:        
        os.system(f"wget 'https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
        os.system(f"7z x '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
        os.system(f"rm '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")

In [80]:
df = pd.DataFrame()

In [81]:
def get_datetime(miliseconds: int):
    return datetime.fromtimestamp(miliseconds)

def create_date_df(df: pd.DataFrame):
    df['date'] = pd.to_datetime(
        df['last_reported'].apply(lambda x: get_datetime(x))
    )
    
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['hour'] = df['date'].dt.hour
    
    return df

In [86]:
for csv in tqdm(os.listdir('./data')):
    new_df = pd.read_csv('./data/' + csv)
    
    # Unique values
    new_df = new_df.drop_duplicates()
    new_df = new_df.dropna(subset=['last_reported', 'last_updated'], axis=0)
    new_df = new_df.sort_values('last_reported', ascending=True)
    
    # Convert some categorical into numerical
    new_df.status = np.where(new_df.status == 'IN_SERVICE', 1, 0)
    new_df.is_charging_station = np.where(new_df.is_charging_station, 1, 0)
    
    # Create the dates from timestamp and group statistics
    new_df = create_date_df(new_df)
    new_df = new_df\
        .groupby(['station_id', 'year', 'month', 'day', 'hour'])\
        .mean(numeric_only=True)
    
    df = pd.concat(
        [df, new_df], 
        axis=0
    )


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [08:50<00:00, 10.61s/it]


In [101]:
# Assume that the NaN in traffic is 0
df.traffic = df.traffic.replace(np.nan, 0)

In [106]:
# Reset which is the index
df = df.reset_index()

In [108]:
df.shape

(16401766, 18)

In [107]:
df.head()

,station_id,year,month,day,hour,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,status,last_updated,ttl,traffic
0,1,2020,5,31,23,9.000000,9.000000,0.0,35.000000,1.0,1.0,1.0,1.590962e+09,1.0,1.0,1.590962e+09,2.000000,0.0
1,1,2020,6,1,0,7.727273,7.727273,0.0,36.272727,1.0,1.0,1.0,1.590964e+09,1.0,1.0,1.590964e+09,17.363636,0.0
2,1,2020,6,1,1,8.076923,8.076923,0.0,35.923077,1.0,1.0,1.0,1.590968e+09,1.0,1.0,1.590968e+09,15.461538,0.0
3,1,2020,6,1,2,7.750000,7.750000,0.0,36.250000,1.0,1.0,1.0,1.590971e+09,1.0,1.0,1.590972e+09,14.083333,0.0
4,1,2020,6,1,3,8.000000,8.000000,0.0,36.000000,1.0,1.0,1.0,1.590975e+09,1.0,1.0,1.590975e+09,14.250000,0.0


### Add station information

In [118]:
def get_station_json(
    url: str = 'https://opendata-ajuntament.barcelona.cat/data/dataset/6aa3416d-ce1a-494d-861b-7bd07f069600/resource/b20e711d-c3bf-4fe5-9cde-4de94c5f588f/download'
):
    res = requests.get(url)
    return res.json()

In [119]:
station_json = get_station_json()

In [ ]:
station_json

{'last_updated': 1684486494,
 'ttl': 4,
 'data': {'stations': [{'station_id': 1,
    'num_bikes_available': 12,
    'num_bikes_available_types': {'mechanical': 11, 'ebike': 1},
    'num_docks_available': 34,
    'last_reported': 1684486321,
    'is_charging_station': True,
    'status': 'IN_SERVICE',
    'is_installed': 1,
    'is_renting': 1,
    'is_returning': 1,
    'traffic': None},
   {'station_id': 2,
    'num_bikes_available': 5,
    'num_bikes_available_types': {'mechanical': 5, 'ebike': 0},
    'num_docks_available': 22,
    'last_reported': 1684486475,
    'is_charging_station': True,
    'status': 'IN_SERVICE',
    'is_installed': 1,
    'is_renting': 1,
    'is_returning': 1,
    'traffic': None},
   {'station_id': 3,
    'num_bikes_available': 0,
    'num_bikes_available_types': {'mechanical': 0, 'ebike': 0},
    'num_docks_available': 27,
    'last_reported': 1684486338,
    'is_charging_station': True,
    'status': 'IN_SERVICE',
    'is_installed': 1,
    'is_renting':

### Add Weather information

### Train / Val / Test Split

In [60]:
df.head()

num_bikes_available  \
station_id year month day hour                        
1          2020 5     31  23               9.000000   
                6     1   0                7.727273   
                          1                8.076923   
                          2                7.750000   
                          3                8.000000   

                                num_bikes_available_types.mechanical  \
station_id year month day hour                                         
1          2020 5     31  23                                9.000000   
                6     1   0                                 7.727273   
                          1                                 8.076923   
                          2                                 7.750000   
                          3                                 8.000000   

                                num_bikes_available_types.ebike  \
station_id year month day hour                                    
1          2020 5     31  23                                0.0   
                6     1   0                                 0.0   
                          1                                 0.0   
                          2                                 0.0   
                          3                                 0.0   

                                num_docks_available  is_installed  is_renting  \
station_id year month day hour                                                  
1          2020 5     31  23              35.000000           1.0         1.0   
                6     1   0               36.272727           1.0         1.0   
                          1               35.923077           1.0         1.0   
                          2               36.250000           1.0         1.0   
                          3               36.000000           1.0         1.0   

                                is_returning  last_reported  \
station_id year month day hour                                
1          2020 5     31  23             1.0   1.590962e+09   
                6     1   0              1.0   1.590964e+09   
                          1              1.0   1.590968e+09   
                          2              1.0   1.590971e+09   
                          3              1.0   1.590975e+09   

                                is_charging_station  last_updated        ttl  
station_id year month day hour                                                
1          2020 5     31  23                    1.0  1.590962e+09   2.000000  
                6     1   0                     1.0  1.590964e+09  17.363636  
                          1                     1.0  1.590968e+09  15.461538  
                          2                     1.0  1.590972e+09  14.083333  
                          3                     1.0  1.590975e+09  14.250000

### Exploration